# The Battle of Neighborhoods
## 1. Introduction/Business Problem


Opening a cafeteria in Toronto

Timeout Company plans to open a cafeteria in the city of Toronto. The decision of where the cafeteira depends highly on the information of the neighborhoods in Toronto (e.g. the most popular avenue, whether there is a park in the neighborhood or not, etc.).

The following information (not limited to) are helpful for making the decision:

* Toronto population
* City demographics
* Are there any venues like parks, shopping centers etc.
* Are there competitors in that location?

## 2. Data

**Neighborhoods in Toronto data source:**
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In order to provide relevant information to Timeout, first I sort the neighborhoods by postal codes, create a table with corresponding longitude and latitude. The next step is to combine these information  with Foursquare API to collect information (venues, competitors, etc.) on the same neighborhoods.

In [1]:
# import libs

import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium
from bs4 import BeautifulSoup
import requests

print('Libraries imported!')

     |████████████████████████████████| 94 kB 4.8 MB/s  eta 0:00:01
Libraries imported!


## 2.1 Download the data

In [2]:
# scrape the table from Wiki
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')
table=soup.find('table')
# convert to pandas dataframe
col_names = ['PostalCode','Borough','Neighborhood']
df = pd.DataFrame(columns = col_names)
# search all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
# data wrangling and cleaning
## drop the data 'Not assigned' Borough
df=df[df['Borough']!='Not assigned']
## deal with 'Not assigned' Neighborhood
df.loc[df['Neighborhood'] =='Not assigned' , 'Neighborhood'] = df['Borough']
# merge neighborhood with the same postal code
temp = df.groupby(['PostalCode','Borough'], sort=False).agg( ', '.join)
df_merge = temp.reset_index()
df_merge.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
# use csv file to get the latitude and longitude
# download and read the csv file
!wget -q -O 'Toronto_long_lat_data.csv'  http://cocl.us/Geospatial_data
df_lon_lat = pd.read_csv('Toronto_long_lat_data.csv')
df_lon_lat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
# merge the latitude and longitude data to neighborhood table
df_lon_lat.columns=['PostalCode','Latitude','Longitude']
df_new = pd.merge(df_merge,
                 df_lon_lat[['PostalCode','Latitude', 'Longitude']],
                 on='PostalCode')
df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


**So far, the geological inforamtion of the neighborhoods are collected.**

## 2.2 Create a map with neighborhoods of Toronto

In [6]:
# Use geopy library to get the latitude and longitude values of Toronto.
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [7]:
# Create a map of Toronto with neighborhoods superimposed on top.
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## 2.3 Combine with Foursquare API

In [1]:
# Define Foursquare Credentials and Version
# information removed

In [9]:
# get the top 100 venues that are in Marble Hill within a radius of 500 meters.
limit = 100
radius = 500
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
toronto_venues = getNearbyVenues(names=df_new['Neighborhood'],
                                   latitudes=df_new['Latitude'],
                                   longitudes=df_new['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [11]:
print('{} venues were returned by Foursquare.'.format(toronto_venues.shape[0]))
toronto_venues.head()

2116 venues were returned by Foursquare.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [12]:
# check how many venues were returned for each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
...,...,...,...,...,...,...
"Willowdale, Willowdale East",34,34,34,34,34,34
"Willowdale, Willowdale West",4,4,4,4,4,4
Woburn,4,4,4,4,4,4


**Till now, the neighborhood geological data is combined with Foursquare API data (combined with venues).**

The next step is to analyze each neighorbood. 

## 2.4 Analyze each neighborhood

In [13]:
# one hot encoding 
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.029412,0.0,0.0,0.0,0.0,0.0
92,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0
93,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0
94,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.125,0.000000,0.0,0.0,0.0,0.0,0.0


In [15]:
# print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                        venue  freq
0              Breakfast Spot  0.25
1                Skating Rink  0.25
2                      Lounge  0.25
3   Latin American Restaurant  0.25
4  Modern European Restaurant  0.00


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.29
1   Coffee Shop  0.14
2  Skating Rink  0.14
3           Pub  0.14
4      Pharmacy  0.14


----Bathurst Manor, Wilson Heights, Downsview North----
          venue  freq
0   Coffee Shop  0.10
1          Bank  0.10
2          Park  0.05
3  Intersection  0.05
4   Bridal Shop  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4                Motel  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.12
1         Coffee Shop  0.08
2      Sandwich Place  0.08
3         Pizza Place  0.08
4        Liquor Store  0.04

In [16]:
# explore top 5 venues of each neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink
1,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Fried Chicken Joint
3,Bayview Village,Chinese Restaurant,Café,Japanese Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Pizza Place,Coffee Shop


In [18]:
# select the neighborhood where the 1st most common venue= Park
neighborhoods_dp = neighborhoods_venues_sorted[neighborhoods_venues_sorted['1st Most Common Venue'] =='Park']
neighborhoods_dp.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
10,Caledonia-Fairbanks,Park,Women's Store,Pool
20,Davisville North,Park,Department Store,Gym / Fitness Center
26,"East Toronto, Broadview North (Old East York)",Park,Intersection,Convenience Store
30,"Forest Hill North & West, Forest Hill Road Park",Park,Sushi Restaurant,Jewelry Store
32,Glencairn,Park,Bakery,Japanese Restaurant


In [19]:
# drop the neighborhood where the 2nd most common venue= Café
neighborhoods_temp = neighborhoods_dp[neighborhoods_dp['2nd Most Common Venue'] !='Café']
neighborhoods_temp.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
10,Caledonia-Fairbanks,Park,Women's Store,Pool
20,Davisville North,Park,Department Store,Gym / Fitness Center
26,"East Toronto, Broadview North (Old East York)",Park,Intersection,Convenience Store
30,"Forest Hill North & West, Forest Hill Road Park",Park,Sushi Restaurant,Jewelry Store
32,Glencairn,Park,Bakery,Japanese Restaurant


In [20]:
# drop the neighborhood where the 2nd most common venue= Café
neighborhoods_dc = neighborhoods_temp[neighborhoods_temp['2nd Most Common Venue'] !='Coffee Shop']
neighborhoods_dc.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
10,Caledonia-Fairbanks,Park,Women's Store,Pool
20,Davisville North,Park,Department Store,Gym / Fitness Center
26,"East Toronto, Broadview North (Old East York)",Park,Intersection,Convenience Store
30,"Forest Hill North & West, Forest Hill Road Park",Park,Sushi Restaurant,Jewelry Store
32,Glencairn,Park,Bakery,Japanese Restaurant


In [21]:
neighborhoods_dc.shape

(8, 4)

In [22]:
# get a full list of selected neighborhoods
neighborhoods_dc.head(8)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
10,Caledonia-Fairbanks,Park,Women's Store,Pool
20,Davisville North,Park,Department Store,Gym / Fitness Center
26,"East Toronto, Broadview North (Old East York)",Park,Intersection,Convenience Store
30,"Forest Hill North & West, Forest Hill Road Park",Park,Sushi Restaurant,Jewelry Store
32,Glencairn,Park,Bakery,Japanese Restaurant
50,"Milliken, Agincourt North, Steeles East, L'Amo...",Park,Playground,Intersection
65,Rosedale,Park,Trail,Playground
90,"Willowdale, Newtonbrook",Park,Women's Store,Distribution Center


In [23]:
# drop the neighborhoods where the 2nd and 3rd most common venues are not related with shopping or entainment activities.
neighborhoods_temp2 = neighborhoods_dc.drop([26, 30 , 32, 50, 65, 90])
neighborhoods_temp2.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
10,Caledonia-Fairbanks,Park,Women's Store,Pool
20,Davisville North,Park,Department Store,Gym / Fitness Center


**The neighborhoods are analyzed, and the new cafeteria is planned for a neighborhood where the most common venue is park, and of which the coffee shop/cafteria venue is not yet saturated (2nd most common venue!=coffee shop or cafe). The locations are further limited with the condition that the 2nd and 3rd should be related with shopping and entainment activites such as sport centers, cinemas, playgrounds, etc. After selection, the short list comes with 2 potential neighborhoods: Caledonia-Fairbanks and Davisville North.**

**Create a map of the selected neighborhoods.**

In [24]:
# create a table of selected two neighborhoods with long and lat
df_new_short = df_new[(df_new["Neighborhood"] == "Caledonia-Fairbanks") | (df_new["Neighborhood"]  == "Davisville North")]
print(df_new_short)

   PostalCode          Borough         Neighborhood   Latitude  Longitude
21        M6E             York  Caledonia-Fairbanks  43.689026 -79.453512
67        M4P  Central Toronto     Davisville North  43.712751 -79.390197


In [25]:
# Create a map of Toronto with selected neighborhoods superimposed on top.
map_toronto_short = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map, all neighborhoods
for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_short)  

# highlight the selected neighborhoods
for lat, lng, borough, neighborhood in zip(df_new_short['Latitude'], df_new_short['Longitude'], df_new_short['Borough'], df_new_short['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#ff0000',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_short)  
    
map_toronto_short

**Compare the population and income of the two selected neighborhoods.**

In [26]:
# Toronto Open Data Catalogue - Neighbourhood Profiles 2016 (CSV)
# https://www.toronto.ca/city-government/data-research-maps/open-data/open-data-catalogue/#8c732154-5012-9afe-d0cd-ba3ffc813d5a

csv_path='https://www.toronto.ca/ext/open_data/catalog/data_set_files/2016_neighbourhood_profiles.csv'
df_pop = pd.read_csv(csv_path,encoding='latin1')
print('Data loaded')

Data loaded


In [27]:
t_neighborhoods = list(df_pop.columns.values)
t_neighborhoods = t_neighborhoods[5:]
dfToronto = pd.DataFrame(index=t_neighborhoods, columns=["Population_2016","Income_2016"])
dfToronto.head()

,Population_2016,Income_2016
Agincourt North,NaN,NaN
Agincourt South-Malvern West,NaN,NaN
Alderwood,NaN,NaN
Annex,NaN,NaN
Banbury-Don Mills,NaN,NaN


In [28]:
# Population_2016 = Population, 2016
# Income_2016 = Total income: Average amount ($)

for index, row in dfToronto.iterrows():
    dfToronto.at[index, 'Population_2016'] = df_pop[index][2]
    dfToronto.at[index, 'Income_2016'] = df_pop[index][2264]
    

df_pi = dfToronto.sort_values('Income_2016')
df_pi.head(20)

,Population_2016,Income_2016
St.Andrew-Windfields,"17,812","100,516"
Edenbridge-Humber Valley,"15,535","101,551"
Lawrence Park North,"14,607","111,730"
Annex,"30,526","112,766"
Yonge-St.Clair,"12,528","114,174"
Bedford Park-Nortown,"23,236","123,077"
Leaside-Bennington,"16,828","125,564"
Kingsway South,"9,271","144,642"
Casa Loma,"10,968","165,047"
Lawrence Park South,"15,179","169,203"


In [74]:
# Creat a table with the population and income of selected neighborhodds Caledonia-Fairbank and Davisville North (officaklly as Mount Pleasant East and Mount Pleasant West)
df_pi_temp = df_pi[(df_pi.index == "Caledonia-Fairbank") | (df_pi.index == "Mount Pleasant East" ) | (df_pi.index == "Mount Pleasant West")]
print(df_pi_temp)

                    Population_2016 Income_2016
Caledonia-Fairbank            9,955      35,112
Mount Pleasant West          29,658      57,039
Mount Pleasant East          16,775      85,340


In [89]:
final = {'Population' : pd.Series([9955, 46433], index=['Caledonia-Fairbanks','Davisville North']), 'Income' : pd.Series([35112, 142379], index=['Caledonia-Fairbanks','Davisville North']),'Per capita income' :pd.Series([3.5271, 3.0663], index=['Caledonia-Fairbanks','Davisville North'])}

In [90]:
df_final = pd.DataFrame(final)
df_final

,Population,Income,Per capita income
Caledonia-Fairbanks,9955,35112,3.5271
Davisville North,46433,142379,3.0663


**Caledonia-Fairbanks is choosen as its PCI (per capita income) is higher than Davisville North.**